# Home Cage Observation Fight Chase Plotting

## Importing other Python Libraries/Modules

In [1]:
import re
import os
import sys
import string
import glob
import ast
from collections import Counter
from collections import defaultdict
import warnings

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

# Summary

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [4]:
protocol_name = "cage"

In [5]:
prefix_name = "hco"

In [6]:
cohort_name = ""

## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [7]:
input_folder = os.path.join(".", "data")

In [8]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = '*[cC]age*'

# NOTE: If there is a set excel file used for this notebook, make the cell below into a code cell and put the path in the "" (quotation marks)

In [9]:
raw_data_file_path = ""

- Asking the user what the path to the recording files are, with the option of using wildcards

In [10]:
if not raw_data_file_path:
    raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    """)

Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    


- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have key word you specified in the file name

In [11]:
if not raw_data_file_path:
    # Using the user inputted path/pattern
    if raw_data_glob_pattern.strip():
        # Getting a list of all matching files
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Checking if there were any files that matched
        if not raw_data_files_list:
            raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

    # Using the default pattern
    else:
        raw_data_glob_pattern = os.path.join(input_folder + "**", raw_data_file_keyword)
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Searching for matching files with recursion
        if not raw_data_files_list:
            raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


In [12]:
raw_data_files_list

['./data/pilot_1_iwata_Home_Cage_Behaviors_Data.xlsx',
 './data/pilot_2_iwata_Home_Cage_Behaviors_Data.xlsx',
 './data/pilot_3_iwata_Home_Cage_Behaviors_Data.xlsx']

## Getting a list of all the sheets

In [13]:
file_name_to_sheet_names = defaultdict(dict)
for file_path in raw_data_files_list:
    # Getting the sheet names for the excel file
    xls = pd.ExcelFile(file_path)
    file_name = os.path.basename(file_path)
    file_name_to_sheet_names[file_name]["sheet_names"] = [sheet for sheet in xls.sheet_names if "CAGE" in sheet]
    file_name_to_sheet_names[file_name]["file_path"] = file_path
    file_name_to_sheet_names[file_name]["pilot"] = file_name.split("_")[1]


- Getting the dataframe for each file

In [14]:
for file_name, value in file_name_to_sheet_names.items():
    value["hco_dataframe"] = {}
    for sheet in value["sheet_names"]:
        if "Unnamed: 0" in pd.read_excel(value["file_path"], sheet_name=sheet).columns:
            value["hco_dataframe"][sheet] = pd.read_excel(value["file_path"], sheet_name=sheet, header=1)
        else:
            value["hco_dataframe"][sheet] = pd.read_excel(value["file_path"], sheet_name=sheet)
        value["hco_dataframe"][sheet]["cage"] = sheet
        value["hco_dataframe"][sheet]["pilot"] = value["pilot"]

In [15]:
value["hco_dataframe"]["CAGE6"]

,observer,notes,length of observations,date,cage #,winner,loser,action,removed_subject_id,cage,pilot
0,Meghan,12:40 - 1:10; a few hours after tube testing,30,2022-09-07,6,NaN,NaN,NaN,NaN,CAGE6,3
1,Meghan,"12:30-1PM; isolated for 45 min, did tube test ...",30,2022-09-08,6,NaN,NaN,NaN,NaN,CAGE6,3
2,Meghan,2:17-2:47PM; after 3 hours of isolation,30,2022-09-14,6,NaN,NaN,NaN,NaN,CAGE6,3
3,Jocelyn,"9:12-9:42pm, nothing",30,2022-09-19,6,NaN,NaN,NaN,NaN,CAGE6,3
4,Naeliz,"start 2:21pm, right after RT, nothing happened...",30,2022-09-21,6,NaN,NaN,NaN,NaN,CAGE6,3
5,Naeliz,"start3:35pm, right after Rt, no before feed, o...",30,2022-09-23,6,NaN,NaN,NaN,NaN,CAGE6,3


## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end
    - One dictionary per sheet

In [16]:
for file_name, file_value in file_name_to_sheet_names.items():
    for sheet, sheet_value in file_value["hco_dataframe"].items():
        # Creating a dictionary that maps the original column name to the standarized one
        column_name_to_standarized = defaultdict(dict)
        for col in sheet_value:
            # Making the column name lower case and removing the spaces
            column_name_to_standarized[col] = "_".join(str(col).lower().strip().split(" "))
        file_name_to_sheet_names[file_name]["hco_dataframe"][sheet] = sheet_value.rename(columns=column_name_to_standarized)

## Combining All Dataframes

In [17]:
all_dataframes = []
for file_name, file_value in file_name_to_sheet_names.items():
    print(file_name)
    for sheet, sheet_value in file_value["hco_dataframe"].items():
        all_dataframes.append(sheet_value)

pilot_1_iwata_Home_Cage_Behaviors_Data.xlsx
pilot_2_iwata_Home_Cage_Behaviors_Data.xlsx
pilot_3_iwata_Home_Cage_Behaviors_Data.xlsx


In [18]:
concatted_all_dataframe = pd.concat(all_dataframes)

In [19]:
concatted_all_dataframe.columns

Index(['oberserver', 'notes', 'length_of_observations', 'date', 'cage_#',
       'winner', 'loser', 'action', 'removed_subject_id', '1', '2', '3', '4',
       'cage', 'pilot', 'unnamed:_9', 'unnamed:_10', 'unnamed:_11',
       'unnamed:_12', 'observer', 'initiator', 'reciever', 'as_count',
       'face_sniffing_count', 'fs_count'],
      dtype='object')

In [20]:
concatted_all_dataframe = concatted_all_dataframe[['notes', 'date', 'winner', 'loser', 'action', 'cage', 'pilot', 'length_of_observations', 'removed_subject_id']]

## Preprocessing the dataframes

- Removing all rows that don't involve HCO
    - Anything tube test related or other ways incomprehensible, I have labled "NA" in removed_subject_id

In [21]:
concatted_all_dataframe[concatted_all_dataframe["removed_subject_id"] == "remove"].head()

,notes,date,winner,loser,action,cage,pilot,length_of_observations,removed_subject_id
172,C57,2022-07-06,1.3,1.2,fs,CAGE1,2,30.0,remove
173,#4 remove,NaT,1.2,1.4,as,CAGE1,2,NaN,remove
174,lots of exploration,NaT,1.1,1.4,fs,CAGE1,2,NaN,remove
175,grooming and bald patches,NaT,1.2,1.1,as,CAGE1,2,NaN,remove
176,NaN,NaT,1.4,1.1,fs,CAGE1,2,NaN,remove


In [22]:
concatted_all_dataframe = concatted_all_dataframe[~(concatted_all_dataframe["removed_subject_id"] == "remove")]

In [23]:
concatted_all_dataframe["removed_subject_id"].unique()

array([nan, 3.1, 4.4, 1.1, 1.2, 2.3, 2.2, 2.1, 4.1, 5.3, 6.1, 4.3, 5.1],
      dtype=object)

- Fixing the actions, like filling in the rows with no actions as "NA"

In [24]:
concatted_all_dataframe["action"].unique()

array(['c', 'f', 'g', 'groom (g)', nan, 'mount (m)', 'm', 'fs', 'ag',
       'as', 'fs ', 'ag ', 'as ', 'f ', 'b', 's'], dtype=object)

In [25]:
concatted_all_dataframe.head()

,notes,date,winner,loser,action,cage,pilot,length_of_observations,removed_subject_id
0,isolated all animals for 30 min. then reintrod...,2022-05-04,3.1,3.4,c,CAGE3,1,30.0,NaN
1,NaN,NaT,3.1,3.4,f,CAGE3,1,NaN,NaN
2,NaN,NaT,3.1,3.2,f,CAGE3,1,NaN,NaN
3,NaN,NaT,3.1,3.4,f,CAGE3,1,NaN,NaN
4,NaN,NaT,3.1,3.2,f,CAGE3,1,NaN,NaN


In [26]:
concatted_all_dataframe["action"] = concatted_all_dataframe["action"].fillna("NA")

In [27]:
concatted_all_dataframe["action"] = concatted_all_dataframe["action"].apply(lambda x: x.strip())

In [28]:
concatted_all_dataframe["action"].unique()

array(['c', 'f', 'g', 'groom (g)', 'NA', 'mount (m)', 'm', 'fs', 'ag',
       'as', 'b', 's'], dtype=object)

- Getting the rows where the dates first appear

In [29]:
concatted_all_dataframe[~(concatted_all_dataframe["date"].isna())].head()

,notes,date,winner,loser,action,cage,pilot,length_of_observations,removed_subject_id
0,isolated all animals for 30 min. then reintrod...,2022-05-04,3.1,3.4,c,CAGE3,1,30.0,NaN
26,"right after tubetesting, and without dominant ...",2022-05-05,3.4,3.3,f,CAGE3,1,30.0,NaN
34,"right after tubetesting, and without dominant ...",2022-05-06,3.2,3.3,g,CAGE3,1,30.0,NaN
39,"powervacuum in cage 3, wasnt able to reintrodu...",2022-05-11,3.3,3.4,c,CAGE3,1,30.0,3.1
51,homecage observation in their regular homecage...,2022-05-13,3.1,3.4,f,CAGE3,1,30.0,NaN


In [30]:
concatted_all_dataframe["new_session"] = False

In [31]:
concatted_all_dataframe.loc[~(concatted_all_dataframe["date"].isna()), 'new_session'] = True

- Getting the session number

In [32]:
concatted_all_dataframe = concatted_all_dataframe.reset_index(drop = True)
concatted_all_dataframe = concatted_all_dataframe.reset_index(drop = False)


In [33]:
first_row_of_each_date = concatted_all_dataframe[~(concatted_all_dataframe["date"].isna())].index

In [34]:
concatted_all_dataframe["session_number"] = concatted_all_dataframe["index"].apply(lambda x: list(first_row_of_each_date).index(x) if x in first_row_of_each_date else np.nan)

- Filling in the missing rows

In [35]:
concatted_all_dataframe["date"] = concatted_all_dataframe["date"].ffill()
concatted_all_dataframe["length_of_observations"] = concatted_all_dataframe["length_of_observations"].ffill()
concatted_all_dataframe["session_number"] = concatted_all_dataframe["session_number"].ffill()

In [36]:
concatted_all_dataframe

,index,notes,date,winner,loser,action,cage,pilot,length_of_observations,removed_subject_id,new_session,session_number
0,0,isolated all animals for 30 min. then reintrod...,2022-05-04,3.1,3.4,c,CAGE3,1,30.0,NaN,True,0.0
1,1,NaN,2022-05-04,3.1,3.4,f,CAGE3,1,30.0,NaN,False,0.0
2,2,NaN,2022-05-04,3.1,3.2,f,CAGE3,1,30.0,NaN,False,0.0
3,3,NaN,2022-05-04,3.1,3.4,f,CAGE3,1,30.0,NaN,False,0.0
4,4,NaN,2022-05-04,3.1,3.2,f,CAGE3,1,30.0,NaN,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2703,2703,"12:30-1PM; isolated for 45 min, did tube test ...",2022-09-08,NaN,NaN,NA,CAGE6,3,30.0,NaN,True,144.0
2704,2704,2:17-2:47PM; after 3 hours of isolation,2022-09-14,NaN,NaN,NA,CAGE6,3,30.0,NaN,True,145.0
2705,2705,"9:12-9:42pm, nothing",2022-09-19,NaN,NaN,NA,CAGE6,3,30.0,NaN,True,146.0
2706,2706,"start 2:21pm, right after RT, nothing happened...",2022-09-21,NaN,NaN,NA,CAGE6,3,30.0,NaN,True,147.0


## Accounting for the power vacuums

In [37]:
concatted_all_dataframe["cage_number"] = concatted_all_dataframe["cage"].apply(lambda x: int(re.search(r'\d+', x).group()))

In [38]:
per_cage_id_suffix = [.1, .2, .3, .4]

In [39]:
concatted_all_dataframe["original_subject_index"] = concatted_all_dataframe["cage_number"].apply(lambda x: [x + num for num in per_cage_id_suffix])

In [40]:
concatted_all_dataframe["current_subject_index"] = concatted_all_dataframe.apply(lambda x: sorted(list(set(x["original_subject_index"]) - set([x["removed_subject_id"]]))), axis=1)

In [41]:
concatted_all_dataframe[~(concatted_all_dataframe["removed_subject_id"].isna())].head()

,index,notes,date,winner,loser,action,cage,pilot,length_of_observations,removed_subject_id,new_session,session_number,cage_number,original_subject_index,current_subject_index
39,39,"powervacuum in cage 3, wasnt able to reintrodu...",2022-05-11,3.3,3.4,c,CAGE3,1,30.0,3.1,True,3.0,3,"[3.1, 3.2, 3.3, 3.4]","[3.2, 3.3, 3.4]"
40,40,NaN,2022-05-11,3.3,3.4,c,CAGE3,1,30.0,3.1,False,3.0,3,"[3.1, 3.2, 3.3, 3.4]","[3.2, 3.3, 3.4]"
41,41,NaN,2022-05-11,3.4,3.2,c,CAGE3,1,30.0,3.1,False,3.0,3,"[3.1, 3.2, 3.3, 3.4]","[3.2, 3.3, 3.4]"
42,42,NaN,2022-05-11,3.3,3.4,c,CAGE3,1,30.0,3.1,False,3.0,3,"[3.1, 3.2, 3.3, 3.4]","[3.2, 3.3, 3.4]"
43,43,NaN,2022-05-11,3.3,3.2,c,CAGE3,1,30.0,3.1,False,3.0,3,"[3.1, 3.2, 3.3, 3.4]","[3.2, 3.3, 3.4]"


In [42]:
groupby_action_count_series = concatted_all_dataframe.groupby(["winner", "action", "session_number", "cage", "pilot"]).count()["index"]

In [43]:
groupby_action_count_df = pd.DataFrame(groupby_action_count_series).reset_index()

In [44]:
groupby_action_count_df

,winner,action,session_number,cage,pilot,index
0,1.1,ag,15.0,CAGE1,2,3
1,1.1,ag,18.0,CAGE1,2,1
2,1.1,ag,20.0,CAGE1,2,1
3,1.1,ag,23.0,CAGE1,2,1
4,1.1,ag,24.0,CAGE1,2,1
...,...,...,...,...,...,...
704,6.4,fs,100.0,CAGE6,2,8
705,6.4,fs,101.0,CAGE6,2,2
706,6.4,fs,103.0,CAGE6,2,1
707,6.4,fs,104.0,CAGE6,2,1


In [45]:

session_to_current_subject_id = pd.Series(concatted_all_dataframe["current_subject_index"].values,index=concatted_all_dataframe["session_number"]).to_dict()
session_to_length_of_observations = pd.Series(concatted_all_dataframe["length_of_observations"].values,index=concatted_all_dataframe["session_number"]).to_dict()

In [46]:
groupby_action_count_df["current_subject_index"] = groupby_action_count_df["session_number"].map(session_to_current_subject_id)
groupby_action_count_df["length_of_observations"] = groupby_action_count_df["session_number"].map(session_to_length_of_observations)

In [50]:
groupby_action_count_df

,winner,action,session_number,cage,pilot,index,current_subject_index,length_of_observations
0,1.1,ag,15.0,CAGE1,2,3,"[1.1, 1.2, 1.3, 1.4]",30.0
1,1.1,ag,18.0,CAGE1,2,1,"[1.1, 1.2, 1.3, 1.4]",30.0
2,1.1,ag,20.0,CAGE1,2,1,"[1.1, 1.2, 1.3, 1.4]",30.0
3,1.1,ag,23.0,CAGE1,2,1,"[1.1, 1.3, 1.4]",30.0
4,1.1,ag,24.0,CAGE1,2,1,"[1.1, 1.2, 1.3, 1.4]",15.0
...,...,...,...,...,...,...,...,...
704,6.4,fs,100.0,CAGE6,2,8,"[6.1, 6.2, 6.3, 6.4]",30.0
705,6.4,fs,101.0,CAGE6,2,2,"[6.2, 6.3, 6.4]",30.0
706,6.4,fs,103.0,CAGE6,2,1,"[6.2, 6.3, 6.4]",30.0
707,6.4,fs,104.0,CAGE6,2,1,"[6.1, 6.2, 6.3, 6.4]",15.0


## Getting the blank values

In [53]:
new_session_dataframe = concatted_all_dataframe[concatted_all_dataframe["new_session"]]

In [61]:
new_session_dataframe["subject_per_15_min"] = new_session_dataframe.apply(lambda x: [(x["pilot"], subj) for subj in x["current_subject_index"]] * int(x["length_of_observations"] / 15), axis=1)

/tmp/ipykernel_21713/2258039612.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_session_dataframe["subject_per_15_min"] = new_session_dataframe.apply(lambda x: [(x["pilot"], subj) for subj in x["current_subject_index"]] * int(x["length_of_observations"] / 15), axis=1)


In [69]:
two_dimensional_all_pilot_and_subject = new_session_dataframe["subject_per_15_min"].to_list()

In [70]:
flattened_all_pilot_and_subject = [item for sublist in two_dimensional_all_pilot_and_subject for item in sublist]

In [73]:
pilot_and_id_to_15min_sess_count = Counter(flattened_all_pilot_and_subject)

In [74]:
pilot_and_id_to_15min_sess_count

Counter({('1', 3.1): 8,
         ('1', 3.2): 10,
         ('1', 3.3): 10,
         ('1', 3.4): 10,
         ('1', 4.1): 15,
         ('1', 4.2): 15,
         ('1', 4.3): 15,
         ('1', 4.4): 9,
         ('2', 1.1): 27,
         ('2', 1.2): 29,
         ('2', 1.3): 31,
         ('2', 1.4): 31,
         ('2', 2.1): 31,
         ('2', 2.2): 31,
         ('2', 2.3): 29,
         ('2', 2.4): 33,
         ('2', 3.1): 28,
         ('2', 3.2): 32,
         ('2', 3.3): 32,
         ('2', 3.4): 32,
         ('2', 4.1): 22,
         ('2', 4.2): 26,
         ('2', 4.3): 26,
         ('2', 4.4): 26,
         ('2', 5.1): 26,
         ('2', 5.2): 26,
         ('2', 5.3): 22,
         ('2', 5.4): 26,
         ('2', 6.1): 22,
         ('2', 6.2): 26,
         ('2', 6.3): 26,
         ('2', 6.4): 26,
         ('3', 1.1): 12,
         ('3', 1.2): 12,
         ('3', 1.3): 12,
         ('3', 1.4): 12,
         ('3', 2.1): 12,
         ('3', 2.2): 12,
         ('3', 2.3): 12,
         ('3', 2.4): 12,
  

## Getting the number of times that a chace or fight occured

In [48]:
raise ValueError()

ValueError: 

In [ ]:
first_row_of_each_date

In [ ]:
filtered_all_dataframe = concatted_all_dataframe[concatted_all_dataframe["action"].isin(["c", "f", "NA"])]

- Removing rows without an action and date
    - Rows can have only a date if there is no fighting for that day. But it needs at least the action or the date.

In [ ]:
filtered_all_dataframe = filtered_all_dataframe[~((filtered_all_dataframe["action"] == "NA") & (filtered_all_dataframe["date"].isna()))]

In [ ]:
filtered_all_dataframe.head()

In [ ]:
filtered_all_dataframe[~(filtered_all_dataframe["date"].isna())]

In [ ]:
raise ValueError()

In [ ]:
filtered_all_dataframe["date"] = filtered_all_dataframe["date"].ffill()

In [ ]:
filtered_all_dataframe

In [ ]:
raise ValueError()

## Dropping All Rows without a Winner

- Because we will be filling all empty cells with the previous value, we must remove all rows that don't have any data. This is usually cells that don't have any winners or losers filled in the row.
    

# **NOTE**: If there is a set column that is used for the winner and loser,  the rows up by session, then enter it in the cell below between the quotation marks. Default is `"winner"` and `"loser"`

In [ ]:
all_winner_id_column = winner_id_column = ""
all_loser_id_column = loser_id_column = ""

In [ ]:
for key, value in sheet_name_to_everything.items():
    if not all_winner_id_column:
        # Asking users to specify which column is the one with the winner's information
        value["winner_id_column"] = input("""Type the name of the column of the subject that has won the interaction.
        i.e. "winner"

        The available columns are: {}

        Note: If left blank, the column with "winner" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))
    
    
        # Looks for columns with "winner" in them if none of inputted
        if not value["winner_id_column"]:
            value["winner_id_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "winner" in col.lower()][0]
    else:
        value["winner_id_column"] = all_winner_id_column
    # Standarizing the column name of the winner IDs
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["winner_id_column"]: "winner"})
    value["winner_id_column"] = "winner"
    
    if not all_loser_id_column:   
        # Asking users to specify which column is the one with the winner's information
        value["loser_id_column"] = input("""Type the name of the column of the subject that has won the interaction.
        i.e. "loser"

        The available columns are: {}

        Note: If left blank, the column with "loser" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))

        # Looks for columns with "loser" in them if none of inputted
        if not value["loser_id_column"]:
            value["loser_id_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "loser" in col.lower()][0]
    else:
        value["loser_id_column"] = all_loser_id_column

    # Standarizing the column name of the loser IDs
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["loser_id_column"]: "loser"})
    value["loser_id_column"] = "loser"
        
    # Dropping all rows that don't have any values in the winner column
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].dropna(subset=value["winner_id_column"])
    # Dropping all rows that don't have any values in the loser column
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].dropna(subset=value["loser_id_column"])
    
    # Getting all the floats from the strings, removing any spaces and other characters
    value["processed_behavior_recording_dataframe"][value["winner_id_column"]] = value["processed_behavior_recording_dataframe"][value["winner_id_column"]].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)
    value["processed_behavior_recording_dataframe"][value["loser_id_column"]] = value["processed_behavior_recording_dataframe"][value["loser_id_column"]].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)

In [ ]:
value["processed_behavior_recording_dataframe"].head()

In [ ]:
value["processed_behavior_recording_dataframe"].tail()

# Finding the rows with ties

- If a trial is a tie, there should be another column that indicates that it is a tie. The points will be counted for less. If there is no column, then none of the trials will be counted as ties.

# NOTE: If there is a set column that denotes whether the match has a winner or not, then replace the `None` with the name of the column with quotation marks

In [ ]:
all_ties_column = None

In [ ]:
for key, value in sheet_name_to_everything.items():
    
    if all_ties_column is None:
        value["ties_column"] = None
    elif all_ties_column:
        value["ties_column"] = all_ties_column
    else:
    
        # Asking users to specify which column is the one with the winner's information
        value["ties_column"] = input("""Type the name of the column that specifies whether or not a tie has occured
        i.e. "tie"

        The available columns are: {}

        Note: If left blank, the column with "ties" in the name will be used as the column. 
        If there are none, then this part will be skipped

        """.format(value["processed_behavior_recording_dataframe"].columns))

        # Looks for columns with "tie" in them if none of inputted
        if not value["ties_column"]:
            if [col for col in value["processed_behavior_recording_dataframe"].columns if "tie" in col.lower()]:
                value["ties_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "tie" in col.lower()][0]
            else: 
                value["ties_column"] = None
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    try:
        # Standarizing the name of the tie column
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["ties_column"]: "match_is_tie"})
        value["ties_column"] = "match_is_tie"
        # Fillling in the tie column with 0s so that we can fill in the other columns with the previous values
        value["processed_behavior_recording_dataframe"][value["ties_column"]] = value["processed_behavior_recording_dataframe"][value["ties_column"]].fillna()
        # Converting all the tie values into bool so we can filter for the cells with ties
        value["processed_behavior_recording_dataframe"][value["ties_column"]] = value["processed_behavior_recording_dataframe"][value["ties_column"]].astype(bool)
    except:
        value["processed_behavior_recording_dataframe"] = current_processed_behavior_recording_dataframe
        value["ties_column"] = None

## Adding the session number

- We are adding the session number to all the trials. The session number is counting the number of recording sessions that have happened up until that trial. Usually, each session in the spreadsheet is divided up by a session's first row having the date filled in. So we will label a new session when a date is filled in.

# NOTE: If there is a set column that divides the rows up by session, then enter it in the cell below between the quotation marks. Default is `"date"`

# TODO: Recheck starting here

In [ ]:
all_session_divider_column = ""

In [ ]:
for key, value in sheet_name_to_everything.items():
    if not all_session_divider_column:
        # Allowing the user to choose the column that indicates a new session
        session_divider_column = input("""Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: {}

        NOTE: If left blank, the column with "date" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))
    else:
        session_divider_column = all_session_divider_column
        
    # Making the column name lowercase and removing any punctuation
    value["session_divider_column"] = session_divider_column.lower().strip('"').strip("'").strip()

    # Looks for columns with "date" in them if none is inputted
    if not value["session_divider_column"]:
        value["session_divider_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "date" in col.lower()][0]

    # Checks if that column is in the dataframe
    if value["session_divider_column"] not in value["processed_behavior_recording_dataframe"].columns:
        print("WARNING: {} is not a column in {}".format(value["session_divider_column"], key))
        warnings.warn("Look at warning from above or below")
        value["session_divider_column"] = None
    
    # Standarizing all the session divider columns so that they are named date
    if value["session_divider_column"]:
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["session_divider_column"]: "date"})
        value["session_divider_column"] = "date"

# TODO: Make this standarized between sheets and get difference for each row

In [ ]:
for key, value in sheet_name_to_everything.items():
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    try:
        value["processed_behavior_recording_dataframe"][value["session_divider_column"]] = value["processed_behavior_recording_dataframe"][value["session_divider_column"]].fillna(method='ffill')

    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")
        value["processed_behavior_recording_dataframe"] = current_processed_behavior_recording_dataframe

In [ ]:
value["processed_behavior_recording_dataframe"].head()

In [ ]:
value["processed_behavior_recording_dataframe"].tail()

# Getting the Session number differences

- Getting the indexes of where each new session starts. So that we can add the session number to each row.

# TODO: Can remove set about not including tie columns

In [ ]:
# Example of the columns that will be kept, removing the ties column
list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))

In [ ]:
# TODO: Add this


In [ ]:
for key, value in sheet_name_to_everything.items():
    # Removing tie columns because not all rows are ties, so we do not want to fill them in
    if value["ties_column"]:
        non_ties_columns = list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))
        value["processed_behavior_recording_dataframe"][non_ties_columns] = value["processed_behavior_recording_dataframe"][non_ties_columns].fillna(method='ffill')
    else:
        # Filling all the empty cells with the value in the previous cell
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].fillna(method='ffill')

    try:
        # Seeing which rows have a different session than the previous one
        # This will be used to plot vertical lines for each new session
        value["processed_behavior_recording_dataframe"]["session_number_difference"] = value["processed_behavior_recording_dataframe"][value["session_divider_column"]].astype('category').cat.codes.diff()

    
    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
value["processed_behavior_recording_dataframe"].head(n=15)

In [ ]:
value["processed_behavior_recording_dataframe"].head(n=15)

In [ ]:
value["processed_behavior_recording_dataframe"].tail(n=15)

In [ ]:
raise ValueError()

## Getting the cage number

- The cage number is sometimes not consistent throughout a spreadsheet for the same cage. So we will try to standarize it into one value.
    - **NOTE**: This assumes cage numbers are actual numbers. And not entirely consisting of letters. If that isn't the case, then you must edit this cell for your needs.

# NOTE: If there is a column that has the cage number information, replace the `None` with the column name in quotation marks

# TODO: Refactor starting here

- Change this when you have winner and loser cage

In [ ]:
cage_num_of_winner_column = None
cage_num_of_loser_column = None

- Change this when you have id to cage

In [ ]:
id_to_cage = {}

In [ ]:
for key, value in sheet_name_to_everything.items():   
    # When you have a dictionary of IDs to Cage Numbers
    if id_to_cage:
        # Specifying the name by default
        value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
        value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
        # Mapping the IDs to the cage number
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = value["processed_behavior_recording_dataframe"][value["winner_id_column"]].map(id_to_cage)
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = value["processed_behavior_recording_dataframe"][value["loser_id_column"]].map(id_to_cage)
        continue
        
    # When you have a column that specifies the cage number
    elif cage_num_of_winner_column and cage_num_of_loser_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
        value["cage_num_of_loser_column"] = cage_num_of_loser_column
    elif cage_num_of_winner_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
    elif cage_num_of_loser_column:
        value["cage_num_of_loser_column"] = cage_num_of_loser_column

    
    if not cage_num_of_winner_column:
        # Asking the user which column has the cage number
        value["cage_num_of_winner_column"] = input("""Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available columns are: {}

        Note: If left blank, the column with "winner" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "winner" and "cage" in its name if no column is specified
        if not value["cage_num_of_winner_column"]:
            # Checking to see if there are any columns with the winner and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "winner" in col.lower()]
            # Using the column with cage in the name if there are none with winner and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
    
    if not cage_num_of_loser_column:
        # Asking the user which column has the cage number
        value["cage_num_of_loser_column"] = input("""Type the name of the column of the cage of the LOSER subject
        i.e. "cage_num_of_loser"

        The available columns are: {}

        Note: If left blank, the column with "loser" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "loser" and "cage" in its name if no column is specified
        if not value["cage_num_of_loser_column"]:
            # Checking to see if there are any columns with the loser and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "loser" in col.lower()]
            # Using the column with cage in the name if there are none with loser and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
    
    # Using the spreadsheet name as the cage number if there are no columns that match
    if value["cage_num_of_winner_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = key

    if value["cage_num_of_loser_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = key       
    
    # Turning the cage names into the float number only
    # Or using the same name if there are no floats
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_winner"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_loser"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)

    

In [ ]:
sheet_name_to_everything["CAGE1"]["processed_behavior_recording_dataframe"]

## Calculating Elo rating

- Example calculation

In [ ]:
calculation.calculate_elo_rating(subject_elo_rating=1000, agent_elo_rating=2000, score=1)

In [ ]:
calculation.update_elo_rating(winner_id="A", loser_id="B")

## Get the Elo rating for all the events

- Going through each row or interaction and calculating the new Elo rating for the winner and loser. This will create a new dataframe based off of the processed behavioral recording dataframe

# NOTE: If there are a set of columns to keep, edit the cell below with the name of the columns each in quotation marks seperated by commas
   - i.e. `['runner', 'date', 'match', 'winner', 'loser', 'notes', 'session_number',
       'session_number_difference']`

In [ ]:
all_columns_to_keep = [""]

In [ ]:
def get_subject_cage_number(x):
    """
    """
    if x["win_draw_loss"] == 1:
        return x["processed_cage_num_of_winner"]
    elif x["win_draw_loss"] == 0:
        return x["processed_cage_num_of_loser"]
    elif x["win_draw_loss"] == 0.5:
        if 1 == pairing_index:
            return x["processed_cage_num_of_winner"] 
        elif 1 == pairing_index:
            return x["processed_cage_num_of_loser"]

In [ ]:
for key, value in sheet_name_to_everything.items():
    if not all_columns_to_keep:
        # Asking the user which columns to carry over to the Elo rating calculation dataframe
        value["columns_to_keep_string"] = input("""Type all the columns that will be copied from the original dataframe to the Elo rating calculated dataframe. 

        All the available actions: {}
        Each column must be put in quotes and seperated by a comma(,). i.e. 'length of observations', 'date', 'cage #'

        NOTE: If left blank, all the columns will be kept
        """.format(value["processed_behavior_recording_dataframe"].columns))


        # Making a list out of the string of inputted sheet names
        if value["columns_to_keep_string"]:
            value["columns_to_keep_string"] = "[" + value["columns_to_keep_string"] + "]"
            value["columns_to_keep_list"] = ast.literal_eval(value["columns_to_keep_string"])
    else: 
        value["columns_to_keep_list"] = all_columns_to_keep
    try:
        # Calculating the Elo rating    
        value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                   winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                             additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])
    
    except:
        nonexistent_columns = set(value["columns_to_keep_list"]) - set(value["processed_behavior_recording_dataframe"].columns)
        print("WARNING: {} does not have {} columns".format(key, nonexistent_columns))
        warnings.warn("Look at warning from above or below")
        # Using all the column names if no column name is specified
        # Removing the winner and loser column because they will be specified as the columns for the Elo rating calculation
        value["columns_to_keep_list"] =  sorted(list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["winner_id_column"]]) - set([value["loser_id_column"]])))
        value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                   winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                             additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])     

    # Making a dataframe from the dictionary 
    value["elo_rating_dataframe"] = pd.DataFrame.from_dict(value["index_to_elo_rating_and_meta_data"], orient="index")
    value["elo_rating_dataframe"]["cage_num_of_subject"] = value["elo_rating_dataframe"].apply(lambda x: get_subject_cage_number(x), axis = 1)
    value["elo_rating_dataframe"]["cage_num_of_agent"] = value["elo_rating_dataframe"].apply(lambda x: get_subject_cage_number(x), axis = 1)


In [ ]:
value["elo_rating_dataframe"]

In [ ]:
value["elo_rating_dataframe"].head()

In [ ]:
value["elo_rating_dataframe"].tail()

In [ ]:
value["elo_rating_dataframe"].groupby("subject_id").count()

## Combining all the Elo rating dataframes into one

In [ ]:
# Putting all the dataframes into one list
all_sheet_elo_scord_dataframe_list = []
for key, value in sheet_name_to_everything.items():    
    all_sheet_elo_scord_dataframe_list.append(value["elo_rating_dataframe"])

# Combining all the dataframes into one
all_sheet_elo_scord_dataframe_combined = pd.concat(all_sheet_elo_scord_dataframe_list)

In [ ]:
all_sheet_elo_scord_dataframe_combined

- Adding the strain information

In [ ]:
all_subject_ids = set(all_sheet_elo_scord_dataframe_combined["subject_id"].unique()).union(set(all_sheet_elo_scord_dataframe_combined["agent_id"].unique()))

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [ ]:
cage_to_strain = {}

In [ ]:
if cage_to_strain:
    all_sheet_elo_scord_dataframe_combined["subject_strain"] = all_sheet_elo_scord_dataframe_combined["cage_num_of_subject"].map(cage_to_strain)
    all_sheet_elo_scord_dataframe_combined["agent_strain"] = all_sheet_elo_scord_dataframe_combined["cage_num_of_agent"].map(cage_to_strain)

- Adding the name of the experiment

In [ ]:
# Adding the name of the experiment
all_sheet_elo_scord_dataframe_combined["experiment_type"] = protocol_name

- Adding the cohort

In [ ]:
all_sheet_elo_scord_dataframe_combined["cohort"] = cohort_name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

In [ ]:
# Checking to see how many rows for each subject in each cage
all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','cage_num_of_subject']).count()

## Creating a dataframe with only the final Elo rating for each subject

In [ ]:
# Checking to see which cage and subject combination has more than one row
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','cage_num_of_subject']).size().reset_index()
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_groupby[all_sheet_elo_scord_dataframe_groupby[0] >= 1]

# Going through each combination and saving the combination to a dictionary
index_to_subject_id_and_processed_cage_number = defaultdict(dict)
for index, row in all_sheet_elo_scord_dataframe_groupby.iterrows():
    index_to_subject_id_and_processed_cage_number[index]['subject_id'] = row['subject_id']
    index_to_subject_id_and_processed_cage_number[index]['cage_num_of_subject'] = row['cage_num_of_subject']


In [ ]:
index_to_subject_id_and_processed_cage_number

- Getting the final Elo rating for each cage and subject combination

In [ ]:
for index, (key, value) in enumerate(index_to_subject_id_and_processed_cage_number.items()):   
    # The Elo rating dataframe for each cage and subject combination
    per_subject_dataframe = all_sheet_elo_scord_dataframe_combined[(all_sheet_elo_scord_dataframe_combined["subject_id"] == value["subject_id"]) & (all_sheet_elo_scord_dataframe_combined["cage_num_of_subject"] == value["cage_num_of_subject"])]
    # Getting the final Elo rating for each combination
    # -1 Means that we're getting the data from the last row

    index_to_subject_id_and_processed_cage_number[index]["final_elo_rating"] = per_subject_dataframe.iloc[-1]["updated_elo_rating"]
    index_to_subject_id_and_processed_cage_number[index]["cohort"] = per_subject_dataframe.iloc[-1]["cohort"]
    try:
        index_to_subject_id_and_processed_cage_number[index]["strain"] = per_subject_dataframe.iloc[-1]["strain"]
    except:
        print("WARNING: {} in cage {} does not have strain information".format(index_to_subject_id_and_processed_cage_number[key]["subject_id"], index_to_subject_id_and_processed_cage_number[key]["cage_num_of_subject"]))
        warnings.warn("Look at warning from above or below")


In [ ]:
id_to_final_elo_rating_df = pd.DataFrame.from_dict(index_to_subject_id_and_processed_cage_number, orient="index")
# Adding protocol name
id_to_final_elo_rating_df["experiment_type"] = protocol_name
# Adding rank
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("cage_num_of_subject")["final_elo_rating"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['cage_num_of_subject', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df.head()

In [ ]:
id_to_final_elo_rating_df.tail()

# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

- Getting the earliest and the latest dates for all the recordings

In [ ]:
all_earlist_dates = []
all_latest_dates = []
for key, value in sheet_name_to_everything.items():
    try:
        # Getting all the earliest dates for each sheet
        all_earlist_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].min())
        all_latest_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].max())
    except:
        print("WARNING: {} does not have dates as columns".format(key, nonexistent_columns))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    # Turning the Dates into a easier to read format
    # Getting the 0th part of split to remove seconds
    earliest_date = str(min(all_earlist_dates)).split()[0]
    latest_date = str(max(all_latest_dates)).split()[0]
    print("Earlist date: {}".format(earliest_date))
    print("Latest date: {}".format(latest_date))
except:
    earliest_date = None
    latest_date = None

- Getting the cage numbers

In [ ]:
all_cages_list = []
# Creating a list of all the cage numbers
for key, value in sheet_name_to_everything.items():
    try:
        for cage in value["elo_rating_dataframe"]["cage_num_of_subject"].unique():
            all_cages_list.append(cage)
    except:
        print("WARNING: {} does not have cage number as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    all_cages_string = "-".join(sorted([sheet.lower().strip("cage").strip() for sheet in all_cages_list]))
    all_cages_string = "cages-{}".format(all_cages_string)
    print("String of cage names to use for file name: {}".format(all_cages_string))
except: 
    warnings.warn("WARNING: There are no cage numbers to make a title out of")
    all_cages_string = None

- Creating an output directory to save the plots

In [ ]:
plot_output_directory = os.path.join(".", "proc", "plots", "{}_elo_rating".format(protocol_name))

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

# **NOTE**: Sometimes this cell needs to be run again to make sure the size is correct

In [ ]:
# Getting the highest and lowest Elo rating for cutoffs of the Y-axis
max_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].max()
min_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].min()

plt.rcParams["figure.figsize"] = (13.5,7.5)
# Making a plot for each sheet
for key, value in sheet_name_to_everything.items():
    # Setting the size of the figure
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_rating_dataframe = value["elo_rating_dataframe"]
    # Using a new figure template for each sheet
    fig, ax = plt.subplots()        
        
    try:
        # Drawing vertical lines that represent when each session begins
        # Based on when a row has a different session than the previous row
        for index, row in elo_rating_dataframe[elo_rating_dataframe['session_number_difference'].astype(bool)].iterrows():
            # Offsetting by 0.5 to avoid drawing the line on the dot
            # Drawing the lines a little above the max and a little below the minimum
            plt.vlines(x=[row["total_match_number"] - 0.5], ymin=min_elo_rating-50, ymax=max_elo_rating+50, colors='black', linestyle='dashed')
    except:
        print("WARNING: {} does not have a column for session divider".format(key))
        warnings.warn("Look at warning from above or below")
            
    # Drawing a line for each subject
    for subject in sorted(elo_rating_dataframe["subject_id"].unique()):
        # Getting all the rows with the current subject
        subject_dataframe = elo_rating_dataframe[elo_rating_dataframe["subject_id"] == subject]
        # Making the current match number the X-Axis
        plt.plot(subject_dataframe["total_match_number"], subject_dataframe["updated_elo_rating"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo rating")
    # Formattnig Cohort and Experiment Name so that it's more readable with spacing and capitalization
    formatted_cohort_name = " ".join(cohort_name.split("_")).capitalize()
    formatted_protocol_name = string.capwords(" ".join(protocol_name.split("_")))
    formatted_cage_name = " ".join((re.match(r"([a-z]+)([0-9]+)", key, re.I).groups())).capitalize()    
    ax.set_title("{} Elo Rating for {} {}".format(formatted_protocol_name, formatted_cohort_name, formatted_cage_name))
    
    # To show the legend
    ax.legend(loc="upper left")
    # Setting the values of the Y-axis
    plt.ylim(min_elo_rating-50, max_elo_rating+50) 
    # Saving the plot
    file_name_parts_separated = [prefix_name, cohort_name, key, earliest_date, latest_date]
    file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])
    
    file_name_full = "elo_rating_{}.png".format(file_name_parts_combined)
    # Removing all the spaces and replacing them with underscores
    file_name_full = "_".join(file_name_full.split(" "))
    plt.savefig(os.path.join(plot_output_directory, file_name_full))
    # Showing the plots
    plt.show()

# TODO: Continue Editing from here

# Saving the Dataframes

- Creating a subfolder to put the Elo rating Spreadsheets

In [ ]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(protocol_name))

In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

- Saving the dataframes to a file

In [ ]:
file_name_parts_separated = [cohort_name, all_cages_string, prefix_name, earliest_date, latest_date]
file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])

file_name_full = "{}_elo-rating-history.csv".format(file_name_parts_combined)
print(file_name_full)
all_sheet_elo_scord_dataframe_combined.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

In [ ]:
file_name_full = "{}_final-elo-rating.csv".format(file_name_parts_combined)
print(file_name_full)
id_to_final_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

# Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_list = []
for key, value in sheet_name_to_everything.items():
    all_processed_behavior_recording_list.append(value["processed_behavior_recording_dataframe"])
    

- Combining all the dataframes from all the cages

In [ ]:
all_processed_behavior_recording_df = pd.concat(all_processed_behavior_recording_list)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting a tuple of the animal IDs to be able to group

# Note: This assumes that all the animal IDs are different across cages and that all IDs are numbers. i.e. there are no duplicate IDs in different cages.

In [ ]:
# Getting the animal IDs from the Match string
# i.e. Getting all the floats and removing all spaces
# Sorting so that the animal IDs are always in the same order
all_processed_behavior_recording_df["animal_id"] =  all_processed_behavior_recording_df.apply(lambda x: sorted([subject_id.lower().strip() for subject_id in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)]))


In [ ]:
# Making a tuple out of the list
# Tuples are used because lists are mutable and can't be grouped with
all_processed_behavior_recording_df["tuple_animal_id"] = all_processed_behavior_recording_df["animal_id"].apply(lambda x: tuple(x))

In [ ]:
all_processed_behavior_recording_df.head()

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
# Getting only the columns that we need
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['runner', 'date', 'match', 'winner', 'loser', 'notes', 'animal_id', 'tuple_animal_id', "processed_cage_number"]]
# Converting the cage number to an integer
all_processed_behavior_recording_df['processed_cage_number'] = all_processed_behavior_recording_df['processed_cage_number'].astype(int)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting the ID of the winner and the loser for each pair with each match

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['loser'].apply(list)

In [ ]:
all_wins_per_pair[:5]

- Making a dataframe of all the winner IDs and all the loser IDs for a given pair

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: prefix_name + "_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

In [ ]:
raise ValueError()

- Adding the cage information

In [ ]:
# Getting the cage number for each pair
dropped_duplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["tuple_animal_id", "processed_cage_number"]].drop_duplicates()

In [ ]:
dropped_duplicate_all_processed_behavior_recording_df.head()

- Creating a dictionary so that we can create a column for the cage number based on the IDs

## TODO: EDIT below

In [ ]:
try:
    pair_to_cage = pd.Series(dropped_duplicate_all_processed_behavior_recording_df["processed_cage_number"].values, index=dropped_duplicate_all_processed_behavior_recording_df["tuple_animal_id"]).to_dict()
    display(pair_to_cage)
except:
    #TODO: 
    pass

In [ ]:
try:
    all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["{}_tuple_animal_id".format(prefix_name)].map(pair_to_cage)
    all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["processed_cage_number"].astype(int).astype(str)
    display(all_competition_per_pair_df.head())
except:
    pass

In [ ]:
all_competition_per_pair_df["cohort"] = cohort_name


- Calculating the overall winner and loser. Also seeing if there is signficant difference in the number of wins to see if one is dominant over the other

In [ ]:
all_competition_per_pair_df["{}_averaged_winner".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_averaged_loser".format(prefix_name)] = all_competition_per_pair_df["{}_loser".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_winner_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_winner".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_loser_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_loser".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_count_difference".format(prefix_name)] = all_competition_per_pair_df["{}_winner_count".format(prefix_name)] - all_competition_per_pair_df["{}_loser_count".format(prefix_name)]
all_competition_per_pair_df["{}_match_count".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: len(x))
all_competition_per_pair_df["{}_percent_win".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner_count".format(prefix_name)] / x["{}_match_count".format(prefix_name)], axis=1)
all_competition_per_pair_df["{}_percentage_tie".format(prefix_name)] = all_competition_per_pair_df["{}_percent_win".format(prefix_name)].apply(lambda x: True if x < 0.75 else False)

In [ ]:
all_competition_per_pair_df

- Saving the competiton pair results dataframe to a file

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, prefix_name, all_cages_string, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:





all_competition_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))